## getElementsByTagNameHierarchy

* `Element.tagName` returns an __UPPERCASE__ string of the tag name
* basically, we use a pre-order traversal and keep track of the tags we've seen so far
* we then see if the queryTags (tags we want to look for) are a subsequence of the tags we've seen so far
***
* actual solution is probably more optimized
* reason being, we don't have to check for the subsequence every time, we just pass in an index that tells us which tag of the queryTags we're looking for right now
    - this is better because as we return from recursive calls, if the first tag of the queryTags is not at the root but somewhere inside the tree, we will reset the index
    - this tells us that the we are looking for the first ancestor of the hierarchy all over again

In [ ]:
// my solution

/**
 * @param {Document} document
 * @param {string} tagNames
 * @return {Array<Element>}
 * 
 * const doc = new DOMParser().parseFromString(
  `<span>
    <span id="foo">
      <span id="bar">Bar</span>
      Foo
    </span>
    <p>Paragraph</p>
    <span id="baz">Baz</span>
    <div>
      <span id="inner">Inner</span>
    </div>
  </span>`,
  'text/html',
);

const elements = doc.querySelectorAll("div span");

elements.forEach(el => {
  console.log(`${el.localName}#${el.id}`);
})

this console logs span#inner which means we have to locate all divs
and check their hierarchies!!!

const doc = new DOMParser().parseFromString(
  `<div>
    <span id="foo">
      <span id="bar">
        Bar
        <span id="bass">Bass</span>
      </span>
      Foo
    </span>
    <p>Paragraph</p>
    <span id="baz">Baz</span>
    <div>
      <span id="inner">Inner</span>
    </div>
  </div>`,
  'text/html',
);

const elements = doc.querySelectorAll("div span span");

elements.forEach(el => {
  console.log(`${el.localName}#${el.id}`);
})

this example prints out span#bar and span#bass
so it is allowed to "skip" and not be direct descendants!!!

what you can do:
0) preorder traversal
1) as you descend, keep track of tag hierarchy in an array
  - using your original tag hierarchy, e.g. ['div', 'span'],
  you can iterate through it and see if all items are in that recursive tag array

 */

const extractTagNames = (tags) => {
  const result = [];
  let current = "";
  for (let i = 0; i < tags.length; i++) {
    const letter = tags[i];
    if (letter !== " ") {
      current += letter;
    } else if (current) {
      result.push(current);
      current = "";
    }
  }

  if (current) {
    result.push(current);
  }

  return result;
}

const isCorrectHierarchy = (queryTags, currentTags) => {
// const tags = ['div', 'span', 'span'];
// const tags2 = ['span', 'div', 'span', 'div', 'div', 'span'];

// reason being, that's the element we're on
// if it doesn't event match, then it should not be regarded!
  if (queryTags.at(-1) !== currentTags.at(-1)) {
    return false;
  }

  let current = queryTags.length - 1;

  for (let i = currentTags.length - 1; i >= 0; i--) {
    if (currentTags[i] === queryTags[current]) {
      current--;
    }

    if (current < 0) {
      return true;
    }
  }

  return false;
}

export default function getElementsByTagNameHierarchy(document, tagNames) {
  if (!tagNames) return;
  const queryTags = extractTagNames(tagNames.trim().toUpperCase());
  const result = [];
  
  const traverse = (el, currentTags) => {
    if (!el) return;

    currentTags.push(el.tagName);

    if (el.tagName && isCorrectHierarchy(queryTags, currentTags)) {
      result.push(el);
    }
    
    for (const child of el.children) {
      traverse(child, currentTags);
    }

    currentTags.pop();
  };

  traverse(document, [])
  return result;
}

In [ ]:
// actual solution

/**
 * @param {Document} document
 * @param {string} tagNames
 * @return {Array<Element>}
 */
export default function getElementsByTagNameHierarchy(document, tagNames) {
  const results = [];
  const tagTokens = tagNames.toUpperCase().trim().split(/\s+/);
  const lastIndex = tagTokens.length - 1;

  if (tagTokens.length === 0) {
    return results;
  }

  function traverse(el, tagTokenIndex) {
    if (el == null) {
      return;
    }

    const currentTagToken = tagTokens[tagTokenIndex];
    const elementMatchesCurrentTag = el.tagName === currentTagToken;
    const isLastTag = tagTokenIndex === lastIndex;

    if (elementMatchesCurrentTag && isLastTag) {
      results.push(el);
    }

    const nextIndex = elementMatchesCurrentTag
      ? Math.min(tagTokenIndex + 1, lastIndex) // So as not to increment past the last index.
      : tagTokenIndex;

    for (const child of el.children) {
      traverse(child, nextIndex);
    }
  }

  traverse(document.body, 0);

  return results;
}
